In [2]:
##############################
# The goal of this code is to combine output from individual exp into 1 file (-4K, ctl, +4K in a single file)
# Individual exp output comes from the two notebooks:
# (1) 0-2.Calculate_KWamplitude_...ipynb
# (2) 0-3.Calculate_laglon_regression...ipynb
# Mu-Ting Chien
# 2024.3.19
#####################################

In [3]:
from netCDF4 import Dataset
import numpy as np
import matplotlib.pyplot as plt
import os
from matplotlib.cm import get_cmap
import cmaps
import sys
sys.path.append('/glade/work/muting/function/')
import KW_diagnostics as KW
import mjo_mean_state_diagnostics as MJO
import create_my_colormap as mycolor
RWB = mycolor.red_white_blue()

/glade/u/ssg/ch/usr/jupyterhub/envs/cmip6-201910a/lib/python3.7/site-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
dir_out = '/glade/work/muting/KW/'
CASENAME_LIST2 = list([ 'SST_AQP3_Qobs_27_-4K','SST_AQP3_Qobs_27','SST_AQP3_Qobs_27_4K'])
CASENAME_SHORT_LIST = list(['-4K','CTL','4K'])
output_dir_all = dir_out+'output_data/All_3hr_10yr/'
SST_max = np.arange(23,35,4)
figdir = dir_out+'figure/Post_general/10yr_new/'
os.makedirs(figdir,exist_ok=True) 
s2d = 86400
PI = '\u03C0'
pi = np.pi
g  = 9.8
re = 6371*1000 #earth radius (m)

title = list(['(a) -4K','(b) CTL','(c) +4K'])

In [4]:
###############################################
# Combine data from individual experiment into 1 single file
######################################################
for icase in range(0,3):

    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    output_dir = dir_out+'output_data/'+CASENAME+'/'

    # Load signal strength in WK diagram
    data = np.load(output_dir+'pr_wavenum_freq.npz')
    if icase == 0:
        x  = data['x']
        y  = data['y'] 
        n0 = np.size(x,0)
        n1 = np.size(x,1)
        raw_sym = np.empty([n0,n1,3])
        r_sym   = np.empty([n0,n1,3])

    raw_sym[:,:,icase]  = data['power_pr']
    r_sym[:,:,icase]    = data['r_sym']


    # Load KW amplitude, growth rate, phase speed
    data = np.load(output_dir+'kw_amplitude_gr_Cp.npz')

    if icase == 0:
        kw_amp  = np.empty([3])
        EAPEGR1 = np.empty([3])
        EAPEGR2 = np.empty([3])
        EKEGR1 = np.empty([3])
        EKEGR2 = np.empty([3])
        Cp1 = np.empty([3])
        Cp2 = np.empty([3])
        Cp  = np.empty([3])
        zwnum_ave = np.empty([3])
        freq_ave  = np.empty([3])
    #
    kw_amp[icase]  = data['KW_amplitude']
    EAPEGR1[icase] = data['EAPEGR1']*s2d
    EAPEGR2[icase] = data['EAPEGR2']*s2d
    EKEGR1[icase]  = data['EKEGR1']*s2d
    EKEGR2[icase]  = data['EKEGR2']*s2d
    Cp1[icase] = data['Cp1']
    Cp2[icase] = data['Cp2']

    # Load KW phase composite precip
    data = np.load(output_dir+'precip_kw.npz')
    if icase == 0:
        phase = data['phase_bin']
        nph   = np.size(phase)
        pr_kw = np.empty([nph, 3])
    pr_kw[:,icase] = data['pr_kw'][:]

    # Load EOF U1, U2
    data = np.load(output_dir+'EOF_WU.npz')
    if icase == 0:
        plev_ref = data['plev_eofu'][:]
        nlev_ref = np.size(plev_ref)
        EOFU1   = np.empty([nlev_ref,3])
        EOFU2   = np.empty([nlev_ref,3])
    EOFU1[:,icase] = data['EOF1_U'][:]
    EOFU2[:,icase] = data['EOF2_U'][:] 


    # Load EOF Q1, Q2
    data = np.load(output_dir+'EOF.npz')
    if icase == 0:
        plev_eof = data['plev_small']
        ilevmin = np.argwhere(plev_eof==np.min(plev_ref)).squeeze()
        ilevmax = np.argwhere(plev_eof==np.max(plev_ref)).squeeze()
        EOFQ1 = np.empty([nlev_ref,3])
        EOFQ2 = np.empty([nlev_ref,3])
    EOFQ1[:,icase] = data['R_EOF1_smooth'][ilevmin:ilevmax+1]
    EOFQ2[:,icase] = data['R_EOF2_smooth'][ilevmin:ilevmax+1]

    # Load KW composite vertical structure: QT
    data = np.load(output_dir+'QT_kw.npz')
    if icase == 0:
        plev_big = data['plev']
        ilevmin = np.argwhere(plev_big==np.min(plev_ref)).squeeze()
        ilevmax = np.argwhere(plev_big==np.max(plev_ref)).squeeze()
        Q_kw  = np.empty([nph, nlev_ref, 3])
        T_kw  = np.empty([nph, nlev_ref, 3])
        Q1_kw  = np.empty([nph, nlev_ref, 3])
        Q2_kw  = np.empty([nph, nlev_ref, 3])
        T1_kw  = np.empty([nph, nlev_ref, 3])
        T2_kw  = np.empty([nph, nlev_ref, 3])

    Q_kw[:,:,icase] = data['Q_KW'][:, ilevmin:ilevmax+1]*s2d
    T_kw[:,:,icase] = data['T_KW'][:, ilevmin:ilevmax+1]

    Q1_kw[:,:,icase], Q2_kw[:,:,icase] = KW.vertical_mode_decomposition(Q_kw[:,:,icase], EOFQ1[:,icase], EOFQ2[:,icase])
    T1_kw[:,:,icase], T2_kw[:,:,icase] = KW.vertical_mode_decomposition(T_kw[:,:,icase], EOFQ1[:,icase], EOFQ2[:,icase])

    # Load KW composite vertical structure: FU
    data = np.load(output_dir+'FU_kw.npz')
    if icase == 0:
        plev_big_FU = data['plev_FU']
        ilevmin = np.argwhere(plev_big_FU==np.min(plev_ref)).squeeze()
        ilevmax = np.argwhere(plev_big_FU==np.max(plev_ref)).squeeze()        
        F_kw   = np.empty([nph, nlev_ref, 3])
        U_kw   = np.empty([nph, nlev_ref, 3])
        F1_kw  = np.empty([nph, nlev_ref, 3])
        F2_kw  = np.empty([nph, nlev_ref, 3])
        U1_kw  = np.empty([nph, nlev_ref, 3])
        U2_kw  = np.empty([nph, nlev_ref, 3])
    F_kw[:,:,icase] = data['F_KW'][:, ilevmin:ilevmax+1]
    U_kw[:,:,icase] = data['U_KW'][:, ilevmin:ilevmax+1]

    F1_kw[:,:,icase], F2_kw[:,:,icase] = KW.vertical_mode_decomposition(F_kw[:,:,icase], EOFU1[:,icase], EOFU2[:,icase])
    U1_kw[:,:,icase], U2_kw[:,:,icase] = KW.vertical_mode_decomposition(U_kw[:,:,icase], EOFU1[:,icase], EOFU2[:,icase])


    # Make sure all vertical levels are the same
    if np.size(F_kw,1)!=nlev_ref or np.size(U_kw,1)!=nlev_ref or np.size(Q_kw,1)!=nlev_ref or np.size(T_kw,1)!=nlev_ref or np.size(EOFQ1,0)!=nlev_ref or np.size(EOFU1,0)!=nlev_ref:
        print('Error: Vertical size mismtaches!')

    # Load LH
    data = np.load(output_dir+'kw_composite_LH.npz')
    phase = data['phase']
    nphase = np.size(phase)
    if icase == 0:
        lh_kw = np.empty([nphase, 3])
    lh_kw[:,icase] = data['lh_ano_kw']


# Load Cp, zwnum, freq calcualted from lag-regression
data = np.load(output_dir_all+'./repetitive/kw_zwnum_freq_Cp_ave_from_precip_lag_regression.npz')
Cp        = data['Cp_ave']
zwnum_ave = data['zwnum_ave']
freq_ave  = data['freq_ave']


    
# Save all data outputs together in 1 file
np.savez(output_dir_all+'pr_spectrum_all.npz', x=x, y=y, raw_sym=raw_sym, r_sym=r_sym)
np.savez(output_dir_all+'kw_amplitude_gr_Cp_all.npz', kw_amp=kw_amp, EAPEGR1=EAPEGR1, EAPEGR2=EAPEGR2, EKEGR1=EKEGR1, EKEGR2=EKEGR2,Cp=Cp, Cp1=Cp1, Cp2=Cp2, zwnum_ave=zwnum_ave, freq_ave=freq_ave)
np.savez(output_dir_all+'kw_composite_3D_QTUF_all.npz',phase=phase, plev_ref=plev_ref, Q_kw=Q_kw, T_kw=T_kw, Q1_kw=Q1_kw, T1_kw=T1_kw, Q2_kw=Q2_kw, T2_kw=T2_kw, F_kw=F_kw, U_kw=U_kw, F1_kw=F1_kw, U1_kw=U1_kw, F2_kw=F2_kw, U2_kw=U2_kw)
np.savez(output_dir_all+'EOF_all.npz',plev_ref=plev_ref, EOFQ1=EOFQ1, EOFQ2=EOFQ2, EOFU1=EOFU1, EOFU2=EOFU2)
np.savez(output_dir_all+'kw_comoposite_2D_LH.npz',phase=phase, lh_kw=lh_kw)

In [17]:
print('Zwnum:',zwnum_ave)
print('Period (day):',freq_ave)
print('Cp (m/s):',Cp)
print('Cp1 (m/s):',Cp1)
print('Cp2 (m/s):',Cp2)
print('KW amp (mm/day):',kw_amp)

Zwnum: [9. 9. 8.]
Period (day): [3.75 3.   2.5 ]
Cp (m/s): [12.93694276 16.14079195 22.14137775]
Cp1 (m/s): [21.38740186 24.03341728 23.36928963]
Cp2 (m/s): [12.95380659 13.54211787 15.03421311]
KW amp (mm/day): [3.26408596 2.85123272 2.1890906 ]


In [18]:
print('Cp increase by ? %/K:',( (Cp[1]-Cp[0])/Cp[1] + (Cp[2]-Cp[1])/Cp[1] )/8)
print('KW amplitude decrease by ? %/K:',( (kw_amp[1]-kw_amp[0])/kw_amp[1] + (kw_amp[2]-kw_amp[1])/kw_amp[1] )/8)

Cp increase by ? %/K: 0.0712823991552948
KW amplitude decrease by ? %/K: -0.04712853457193346


In [19]:
# 2. Save mean state vertical profile of temperature
for icase in range(0,3):
    
    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    output_dir = dir_out+'output_data/'+CASENAME+'/'
    data = np.load(output_dir+'QT.npz')
    plev_T = data['plev']
    ilev_T_min = np.argwhere(plev_T==np.min(plev_ref)).squeeze()
    ilev_T_max = np.argwhere(plev_T==np.max(plev_ref)).squeeze()
    plev_T = plev_T[ilev_T_min: ilev_T_max+1]
    T = data['Tproj'][:,ilev_T_min:ilev_T_max+1,:] #(time, plev, lon)
    
    if icase == 0:
        Tm = np.empty([np.size(plev_ref),3])
        ilev_melt = np.zeros([3]).astype('int')
    Tm[:,icase] = np.mean(np.mean(T,2),0)
np.savez(output_dir_all+'Tm_all.npz',Tm=Tm, plev_ref=plev_ref)

In [21]:
# 3. Combine KW composite q vertical profile into 1 file
for icase in range(0,3):

    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    output_dir = dir_out+'output_data/'+CASENAME+'/'
    data = np.load(output_dir+'kw_composite_rh_qv_phase_plev.npz')#,rh_kw=rh_kw, qv_kw=qv_kw, phase=phase, rh=rh, rh_ano=rh_ano, qv=qv, qv_ano=qv_ano, time=time, plev=plev, lon=lon)
    plev = data['plev']
    qv_kw = data['qv_kw'] #(phase, plev)
    phase = data['phase']
    nlev = np.size(plev)

    if icase == 0:
        phase = data['phase']
        nphase = np.size(phase)
        qv_kw_all = np.empty([nphase, nlev, 3]) # all level

    qv_kw_all[:,:,icase] = qv_kw*1000
np.savez(output_dir_all+'kw_composite_3D_q_all.npz', qv_kw_all=qv_kw_all, plev=plev, phase=phase)

In [23]:
# 4. Calculate Cp1 and Cp2 with 1 parameter varying each time
Lz1 = np.empty([3])
Lz2 = np.empty([3])
N2_1 = np.empty([3])
N2_2 = np.empty([3])
a1 = np.empty([3])
a2 = np.empty([3])
Hs = np.empty([3])
mm1 = np.empty([3])
mm2 = np.empty([3])
Rd = 287
g = 9.8
for icase in range(0,3):
    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    output_dir = dir_out+'output_data/'+CASENAME+'/'
    data = np.load(output_dir+'Cp_updated.npz')
    Lz1[icase] = data['Lz1']
    Lz2[icase] = data['Lz2']
    N2_1[icase] = data['N2_1']
    N2_2[icase] = data['N2_2']
    a1[icase] = data['a1']
    a2[icase] = data['a2']

    data = np.load(output_dir+'QT.npz')
    Tm = np.nanmean(data['Tm']) # vertical average
    Hs[icase] = Rd*Tm/g

    mm1[icase] = 2*np.pi/Lz1[icase]
    mm2[icase] = 2*np.pi/Lz2[icase]

# Change Cp1, CP2 by varying one parameter aT a time (_N means only N varies, the other terms is the same as CTL)
He1_N = N2_1*(1-a1[1])/((mm1[1]**2+1/(4*Hs[1]**2))*g)
He2_N = N2_2*(1-a2[1])/((mm2[1]**2+1/(4*Hs[1]**2))*g)

He1_m = N2_1[1]*(1-a1[1])/((mm1**2+1/(4*Hs[1]**2))*g)
He2_m = N2_2[1]*(1-a2[1])/((mm2**2+1/(4*Hs[1]**2))*g)

He1_a = N2_1[1]*(1-a1)/((mm1[1]**2+1/(4*Hs[1]**2))*g)
He2_a = N2_2[1]*(1-a2)/((mm2[1]**2+1/(4*Hs[1]**2))*g)

Cp1_N = (g*He1_N)**0.5
Cp2_N = (g*He2_N)**0.5
Cp1_m = (g*He1_m)**0.5
Cp2_m = (g*He2_m)**0.5
Cp1_a = (g*He1_a)**0.5
Cp2_a = (g*He2_a)**0.5
np.savez(output_dir_all+'Cp_change1parameter_N_m_alpha.npz',\
         Cp1_N=Cp1_N, Cp2_N=Cp2_N, \
         Cp1_m=Cp1_m, Cp2_m=Cp2_m, \
         Cp1_a=Cp1_a, Cp2_a=Cp2_a)

In [8]:
# 5. Load PW and PR and save both variables into 1 file (use in supplementary figure)
iyr_min = 2
iyr_max = 11
for icase in range(0,3):

    CASENAME = CASENAME_LIST2[icase]+'_3h_20y'
    CASENAME_SHORT = CASENAME_SHORT_LIST[icase]+'_20y_3h_20y'

    if icase !=0:
        CASENAME_org = CASENAME
    else:
        CASENAME_org = CASENAME+'_new2'
    
    # (0) Get nfile_skip
    nfile_skip = KW.find_nfile_skip(CASENAME, CASENAME_SHORT, iyr_min, iyr_max)

    # (1) Load pw
    pw, time, lat, lon = KW.load_2D_data_as_1variable(CASENAME_org, 'TMQ','', iyr_min, iyr_max, 90, \
                                                      1, nfile_skip, 0) #unit: kg/m2=mm
    print('finish loading pw')

    # (1.5) Calculate time mean zonal mean
    if icase == 0:
        nlat = np.size(lat)
        pw_zm = np.empty([nlat,3])
    pw_zm[:,icase] = np.mean(np.mean(pw,2),0)
    del pw

    # Load precip
    output_dir = dir_out+'output_data/'+CASENAME+'/'
    data = np.load(output_dir+'precip_zm.npz')
    if icase == 0:
        lat = data['lat']
        nlat = np.size(lat)
        pr_zm = np.empty([nlat,3])
    pr_zm[:,icase] = data['pr_zm']
    print('finish loading pr')

    # Save pw and precip
    np.savez(output_dir_all+'mean_state_pr_pw_all.npz', pw_zm = pw_zm, pr_zm=pr_zm, lat=lat)

finish loading pw
finish loading pr
Skip this file, size of time not 31, YEAR: 0007 , MON: 02 , FILE: 00
finish loading pw
finish loading pr
finish loading pw
finish loading pr
